In [12]:
!pip install keyboard

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
from PIL import ImageGrab
import cv2
import win32gui
from digit_detector import digit_detector
from time import sleep
import keyboard
from direct_input import keyCodes,PressKey,ReleaseKey

In [27]:
def drawText(tx,ty,text):
    hdc=win32gui.GetDC(0)
    hwnd=win32gui.WindowFromPoint((0,0))
    win32gui.DrawText(hdc,text, -1, (tx,ty,tx+200,ty+200),0)
    win32gui.ReleaseDC(hwnd,hdc)
    
def drawLine(coords):
    hdc=win32gui.GetDC(0)
    hwnd=win32gui.WindowFromPoint((0,0))
    win32gui.MoveToEx(hdc,coords[0],coords[1])
    win32gui.LineTo(hdc,coords[2],coords[3])
    win32gui.ReleaseDC(hwnd,hdc)

def getScreenShot(bbox):
    return np.array(ImageGrab.grab(bbox))

def getSpeedShot(img):
    return (img[987:1019,150:205].mean(axis =2) > 240).astype(np.int8)

def getMapShot(img):
    return img[795:1051,1627:1883]

def getSpeedDigits(full_img):
    img = (full_img[987:1019,150:205].mean(axis =2) > 240).astype(np.int8)
    return [img[:,:16],img[:,17:33],img[:,35:51]]

def getSpeed(full_img):
    
    return ("".join(digit_detector.predict(np.array([d.flatten() for d in getSpeedDigits(full_img)]))))



In [28]:
def roi(img, vertices):
    #blank mask:
    mask = np.zeros_like(img)
    # fill the mask
    cv2.fillPoly(mask, vertices, 255)
    # now only show the area that is the mask
    masked = cv2.bitwise_and(img, mask)
    return masked

def erase_region(img,vertices):
    #blank mask:
    mask = np.zeros_like(img)
    mask = mask + 255
    # fill the mask
    cv2.fillPoly(mask, vertices, 0)
    # now only show the area that is the mask
    masked = cv2.bitwise_and(img, mask)
    return masked
    

def draw_lines(img,lines):
    for line in lines:
        coords = line[0]
        cv2.line(img, (coords[0], coords[1]), (coords[2], coords[3]), [255,255,255], 3)

def process_img(image):

    image = cv2.bilateralFilter(image,5,75,75)
 
    processed_img =  cv2.Canny(image, threshold1 = 100, threshold2=100)
    
    h,w = image.shape[:2]
    
    vertices = np.array([[0,h*800/1080],
                         [w*750/1920,h*300/1080],
                         [w*1200/1920,h*300/1080],
                         [w*1600/1920,h*800/1080],
                         [w*1600/1920,h],
                         [0,h]], np.int32)
    
    car = np.array([[w*500/1920,h],[750,550],[1250,550],[1250,h]],np.int32)
    
    gui = np.array([[0,900],[300,900],[300,1080],[0,1080]],np.int32)
    
    processed_img = roi(processed_img, [vertices])
    processed_img = erase_region(processed_img, [car])
    processed_img = erase_region(processed_img,[gui] )
    
    lines = cv2.HoughLinesP(processed_img, 5, np.pi/180, 180,      20,         2)
    
    processed_img = np.zeros_like(processed_img)
    
    draw_lines(processed_img,lines)
    
    lines = cv2.HoughLinesP(processed_img, 2, np.pi/180, 180,      20,         15)
     
    processed_img = np.zeros_like(processed_img)
    
    draw_lines(processed_img,lines)
    
    return processed_img,lines

In [29]:
active = True
keepspeed = False

keepspeed_max = 40

trottle = False

while(active):
    img = getScreenShot((0,0,1920,1080))
    speed = getSpeed(img)
    if(keyboard.is_pressed('F6')):
        active = False
    
    if(keyboard.is_pressed('F7')):
        keepspeed = not keepspeed
        drawText(1800,300,f'keepspeed = {keepspeed}')
        sleep(0.5)
    
    if(keyboard.is_pressed('+')):
        keepspeed_max += 1
        drawText(1700,300,f'keepspeed_max = {keepspeed_max}')
        sleep(0.2)
        
    if(keyboard.is_pressed('-')):
        keepspeed_max += -1
        drawText(1700,300,f'keepspeed_max = {keepspeed_max}')
        sleep(0.2)
        
    if speed != '   ':
        if(int(speed) < keepspeed_max and keepspeed):
            if not trottle:
                PressKey(keyCodes['W'])
                trottle = True
        else:
            if trottle:
                ReleaseKey(keyCodes['W'])
                trottle = False

    processed,lines = process_img(img)
    for line in lines[:,0,:]:
        drawLine(line)
        
    drawText(1800,300,str(speed))
#     sleep(0.0125)
    

KeyboardInterrupt: 

In [20]:
lines[:,0,:]

array([[ 396, 1041,  396,  534],
       [ 499, 1040,  499,  466],
       [1334, 1040, 1380, 1040],
       [ 380, 1041,  380,  545],
       [1508,  966, 1508,  683],
       [ 401,  855,  401,  549],
       [1499,  952, 1499,  676],
       [ 384, 1039,  384,  545],
       [1505,  960, 1505,  679],
       [1417, 1041, 1433, 1041],
       [1520,  956, 1520,  700],
       [1335, 1041, 1379, 1041],
       [ 828,  545,  865,  545],
       [1440, 1040, 1460, 1040],
       [ 498, 1041,  498,  467],
       [ 502, 1042,  502,  467],
       [ 851,  537,  889,  537],
       [1333, 1039, 1381, 1039],
       [ 398, 1039,  398,  536],
       [ 382, 1041,  382,  543],
       [ 634,  545,  650,  545],
       [ 394, 1039,  394,  544],
       [ 736,  545,  765,  545],
       [1500,  952, 1504,  682],
       [ 321, 1041,  346, 1041],
       [ 723,  537,  743,  537],
       [ 410, 1039,  433, 1039],
       [ 867,  545,  888,  545],
       [ 751,  537,  789,  537],
       [ 299, 1039,  346, 1039],
       [ 6

In [11]:
%%timeit
drawText(1800,300,str(123))

107 µs ± 926 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
